In [1]:
import pandas
from scipy import stats
import numpy
pandas.set_option('display.max_columns', None)

In [2]:
data = pandas.read_csv("./data/all_consolidated_raw.csv")

In [3]:
data.sample(3)

,player_season_team,2P%,2PA_per_100poss,2PA_per_36min,2PA_per_game,2PA_totals,2P_per_100poss,2P_per_36min,2P_per_game,2P_totals,3P%,3PAR_advanced,3PA_per_100poss,3PA_per_36min,3PA_per_game,3PA_totals,3P_per_100poss,3P_per_36min,3P_per_game,3P_totals,AGE,AST%_advanced,AST_per_100poss,AST_per_36min,AST_per_game,AST_totals,BLK%_advanced,BLK_per_100poss,BLK_per_36min,BLK_per_game,BLK_totals,BPM_advanced,DBPM_advanced,DRB%_advanced,DRB_per_100poss,DRB_per_36min,DRB_per_game,DRB_totals,DRTG_per_100poss,DWS_advanced,EFG%_per_game,EFG%_totals,EFG_totals,FG%,FGA_per_100poss,FGA_per_36min,FGA_per_game,FGA_totals,FG_per_100poss,FG_per_36min,FG_per_game,FG_totals,FT%,FT.1_per_100poss,FT.1_totals,FTA_per_100poss,FTA_per_36min,FTA_per_game,FTA_totals,FTR_advanced,FT_per_100poss,FT_per_36min,FT_per_game,FT_totals,G,GS,MP_advanced,MP_per_100poss,MP_per_36min,MP_per_game,MP_totals,OBPM_advanced,ORB%_advanced,ORB_per_100poss,ORB_per_36min,ORB_per_game,ORB_totals,ORTG_per_100poss,OWS_advanced,PER_advanced,PF_per_100poss,PF_per_36min,PF_per_game,PF_totals,PLAYER,POS,PTS_per_100poss,PTS_per_36min,PTS_per_game,PTS_totals,SEASON,STL%_advanced,STL_per_100poss,STL_per_36min,STL_per_game,STL_totals,TEAM,TOV%_advanced,TOV_per_100poss,TOV_per_36min,TOV_per_game,TOV_totals,TRB%_advanced,TRB_per_100poss,TRB_per_36min,TRB_per_game,TRB_totals,TS%_advanced,USG%_advanced,VORP_advanced,WS/48_advanced,WS_advanced,MVP_VOTES_SHARE,MVP_WINNER,MVP_PODIUM,MVP_CANDIDATE,TEAM_SEASON,W,L,W/L%,GB,PW,PL,PS/G,PA/G,CONF,CONF_RANK
14,RudyTomjanovich_1974_HOU,0.536,20.6,16.4,18.4,1470,11.0,8.8,9.9,788,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25,11.9,3.5,2.8,3.1,250,1.1,0.9,0.7,0.8,66,3.2,-0.9,16.0,6.8,5.4,6.1,487,101.0,2.1,0.536,0.536,NaN,0.536,20.6,16.4,18.4,1470,11.0,8.8,9.9,788,0.848,NaN,NaN,6.4,5.1,5.7,454,0.309,5.4,4.3,4.8,385,80,NaN,3227,3227,3227,40.3,3227,4.1,7.7,3.2,2.6,2.9,230,NaN,10.8,20.5,3.2,2.6,2.9,230,RudyTomjanovich,SF,27.5,21.9,24.5,1961,1974,1.2,1.2,1.0,1.1,89,HOU,NaN,NaN,NaN,NaN,NaN,11.9,10.0,8.0,9.0,717,0.587,NaN,4.2,0.191,12.8,0.001,False,False,True,HOU_1974,32.0,50.0,0.390,15.0,41.0,41.0,107.4,107.6,EASTERN_CONF,6
6587,HaroldMiner_1994_MIA,0.475,19.8,13.9,8.3,526,9.4,6.6,4.0,250,0.667,0.011,0.2,0.2,0.1,6.0,0.2,0.1,0.1,4.0,22,11.3,3.6,2.5,1.5,95,0.7,0.5,0.3,0.2,13,-1.6,-1.4,6.8,3.1,2.1,1.3,81,110.0,0.7,0.481,0.481,NaN,0.477,20.0,14.1,8.4,532,9.6,6.7,4.0,254,0.828,NaN,NaN,6.8,4.8,2.9,180,0.338,5.6,3.9,2.4,149,63,31.0,1358,1358,1358,21.6,1358,-0.2,6.2,2.8,2.0,1.2,75,109.0,1.9,14.0,5.0,3.5,2.1,132,HaroldMiner,SG,24.9,17.5,10.5,661,1994,1.2,1.2,0.8,0.5,31,MIA,13.5,3.6,2.5,1.5,95.0,6.5,5.9,4.1,2.5,156,0.541,22.4,0.1,0.094,2.7,0.000,False,False,False,MIA_1994,42.0,40.0,0.512,15.0,48.0,34.0,103.4,100.7,EASTERN_CONF,8
2101,FrankieSanders_1981_KCK,0.442,19.6,14.9,3.3,77,8.7,6.6,1.5,34,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24,12.7,4.3,3.3,0.7,17,0.3,0.3,0.2,0.0,1,-2.4,0.9,8.9,3.8,2.9,0.7,15,102.0,0.3,0.442,0.442,NaN,0.442,19.6,14.9,3.3,77,8.7,6.6,1.5,34,0.909,NaN,NaN,5.6,4.3,1.0,22,0.286,5.1,3.9,0.9,20,23,NaN,186,186,186,8.1,186,-3.3,3.6,1.5,1.2,0.3,6,92.0,-0.1,13.0,5.1,3.9,0.9,20,FrankieSanders,SF,22.4,17.0,3.8,88,1981,4.1,4.1,3.1,0.7,16,KCK,19.5,5.3,4.1,0.9,21.0,6.3,5.3,4.1,0.9,21,0.508,24.0,0.0,0.035,0.1,0.000,False,False,False,KCK_1981,40.0,42.0,0.488,12.0,41.0,41.0,106.9,106.9,WESTERN_CONF,6


## 1. Fill NAN

In [4]:
len(data)

20176

In [5]:
def safe_drop(dataframe, cols):
    cols = [col for col in cols if col in dataframe.columns]
    dataframe = dataframe.drop(cols, axis='columns', inplace=False)
    return dataframe

In [6]:
def drop_columns_by_nan_ratio(dataframe, ratio:float):
    if not 0. <= ratio <= 1.:
        raise ValueError("Ratio must be between 0 and 1")
    else:
        ratios = data.isna().sum(axis = 'index') / len(data)
        ratios = ratios[ratios > ratio]
        cols = ratios.index.tolist()
        print("Dropping", cols, "...")
        dataframe = dataframe.drop(cols, axis='columns', inplace=False)
        return dataframe

In [7]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

FT.1_per_100poss    19646
FT.1_totals         19646
EFG_totals          19611
GS                   2220
TOV%_advanced        1075
ORTG_per_100poss     1058
TOV_per_100poss      1008
USG%_advanced        1008
TOV_per_36min        1008
TOV_totals           1003
dtype: int64

In [8]:
data = drop_columns_by_nan_ratio(data, 0.1)

Dropping ['EFG_totals', 'FT.1_per_100poss', 'FT.1_totals', 'GS'] ...


In [9]:
data["MVP_PODIUM"] = data["MVP_PODIUM"].fillna(False)
data["MVP_CANDIDATE"] = data["MVP_CANDIDATE"].fillna(False)
data["MVP_WINNER"] = data["MVP_WINNER"].fillna(False)
data["MVP_VOTES_SHARE"] = data["MVP_VOTES_SHARE"].fillna(0.)

In [10]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

TOV%_advanced       1075
ORTG_per_100poss    1058
USG%_advanced       1008
TOV_per_36min       1008
TOV_per_100poss     1008
TOV_totals          1003
TOV_per_game        1003
FT%                  861
EFG%_totals          666
2P%                  158
dtype: int64

In [11]:
cols_with_nan_all = data.isna().sum(axis = 'index').sort_values(ascending=False)
cols_with_nan_all = cols_with_nan_all[cols_with_nan_all>0]
cols_with_nan_all = cols_with_nan_all.index.tolist()

mvp_candidates = data[data["MVP_CANDIDATE"]==True]

cols_with_nan_candidates = mvp_candidates.isna().sum(axis = 'index').sort_values(ascending=False)
cols_with_nan_candidates = cols_with_nan_candidates[cols_with_nan_candidates>0]
cols_with_nan_candidates = cols_with_nan_candidates.index.tolist()

for col in cols_with_nan_all:
    if col not in cols_with_nan_candidates:
        print("Dropping NAN on col", col, "...")
        data = data.dropna(subset=[col], axis='index')
    else:
        print("Not dropping NAN on col", col, "because some MVPs candidates have NAN too")

Not dropping NAN on col TOV%_advanced because some MVPs candidates have NAN too
Not dropping NAN on col ORTG_per_100poss because some MVPs candidates have NAN too
Not dropping NAN on col USG%_advanced because some MVPs candidates have NAN too
Not dropping NAN on col TOV_per_36min because some MVPs candidates have NAN too
Not dropping NAN on col TOV_per_100poss because some MVPs candidates have NAN too
Not dropping NAN on col TOV_totals because some MVPs candidates have NAN too
Not dropping NAN on col TOV_per_game because some MVPs candidates have NAN too
Dropping NAN on col FT% ...
Dropping NAN on col EFG%_totals ...
Dropping NAN on col 2P% ...
Dropping NAN on col FTR_advanced ...
Dropping NAN on col EFG%_per_game ...
Dropping NAN on col FG% ...
Dropping NAN on col TS%_advanced ...
Dropping NAN on col ORB_per_100poss ...
Dropping NAN on col BLK_per_36min ...
Dropping NAN on col PER_advanced ...
Dropping NAN on col PF_per_100poss ...
Dropping NAN on col PF_per_36min ...
Dropping NAN on 

In [12]:
data.isna().sum(axis = 'index').sort_values(ascending=False).head(10)

TOV_per_game        982
ORTG_per_100poss    982
TOV%_advanced       982
TOV_per_100poss     982
TOV_per_36min       982
USG%_advanced       982
TOV_totals          982
DRB_per_game          0
DWS_advanced          0
DRTG_per_100poss      0
dtype: int64

In [13]:
data[data.isna().any(axis="columns")].SEASON.unique()

array([1974, 1975, 1976, 1977], dtype=int64)

In [14]:
#We will infer values intead of removing years
#data = data[data.SEASON > 1977]

In [15]:
cols_to_fill = data.isna().sum(axis = 'index')
cols_to_fill = cols_to_fill[cols_to_fill>0]
cols_to_fill = cols_to_fill.index.tolist()

# Mean of the same post depending on MVP caliber

for col in cols_to_fill:
    print("Filling", col, "...")
    for is_mvp_candidate in True, False:
        for position in data.POS.unique():
            reference = data[(data["MVP_CANDIDATE"]==is_mvp_candidate) & (data["POS"]==position)][col]
            reference_mean = reference[reference.notna()].mean()

Filling ORTG_per_100poss ...
Filling TOV%_advanced ...
Filling TOV_per_100poss ...
Filling TOV_per_36min ...
Filling TOV_per_game ...
Filling TOV_totals ...
Filling USG%_advanced ...


## 2. Remove outliers

In [16]:
def pretty_print_df_values(dataframe_values):
    entries = []
    for v in dataframe_values:
        entry = " :: ".join([str(vi) for vi in v])
        entries.append(entry)
    print("\n".join(entries))

In [17]:
len(data)

18761

In [18]:
for season in data.SEASON.unique():
    for col in data.columns:
        if col not in ["MVP_VOTES_SHARE", "MVP_WINNER", "MVP_PODIUM", "MVP_CANDIDATE"]:
            try:
                outliers = data[(stats.zscore(data[col]) > 7) & (data.SEASON==season)][["player_season_team", col, "MVP_CANDIDATE"]]
                outliers_mvp_candidates = outliers[outliers["MVP_CANDIDATE"]==True][["player_season_team", col]]
                outliers_non_mvp_candidates = outliers[outliers["MVP_CANDIDATE"]==False][["player_season_team", col]]
                if len(outliers) > 0:
                    print()
                    print("Outliers for", col, "in season", season, ":")
                    pretty_print_df_values(outliers.values)
                if len(outliers_non_mvp_candidates) > 0:
                    invalid_id = outliers_non_mvp_candidates.index.tolist()
                    data = data.loc[~data.index.isin(invalid_id)]
            except TypeError:
                pass


Outliers for BLK_per_game in season 1974 :
ElmoreSmith_1974_LAL :: 4.9 :: False

Outliers for FTR_advanced in season 1974 :
TomPatterson_1974_CAP :: 2.0 :: False

Outliers for TS%_advanced in season 1975 :
WaltBellamy_1975_NOJ :: 1.042 :: False

Outliers for BLK_per_game in season 1976 :
KareemAbdulJabbar_1976_LAL :: 4.1 :: True

Outliers for BLK_totals in season 1976 :
KareemAbdulJabbar_1976_LAL :: 338 :: True

Outliers for FTA_per_36min in season 1976 :
WaltWesley_1976_LAL :: 20.6 :: False

Outliers for FTA_per_100poss in season 1977 :
RudyHackett_1977_NYN :: 28.9 :: False
EarlWilliams_1977_NYN :: 39.7 :: False

Outliers for FTR_advanced in season 1978 :
JimArd_1978_BOS :: 2.0 :: False

Outliers for AST_totals in season 1979 :
KevinPorter_1979_DET :: 1099 :: False

Outliers for BLK_totals in season 1979 :
KareemAbdulJabbar_1979_LAL :: 316 :: True

Outliers for ORB_totals in season 1979 :
MosesMalone_1979_HOU :: 587 :: True

Outliers for STL%_advanced in season 1979 :
GusGerard_1979_

In [19]:
len(data)

18590

## 3. Remove duplicated columns

In [26]:
data.T[data.T.duplicated()].T.sample(5)

,EFG%_totals,MP_totals
0,0.459,2459
1,0.512,1075
2,0.458,1007
3,0.451,238
4,0.491,361


In [24]:
data = data.drop(['MP_per_100poss', 'MP_per_36min'], axis='columns')

In [ ]:
#data = data.T.drop_duplicates(how="all", keep="first").T

## Export

In [28]:
data.to_csv("./data/all_consolidated_final", index=False)